# Disease Diagnose

Nosso trabalho consiste em auxiliar o médico na hora de diagnosticar seus
pacientes.  
Através de um banco de dados, que será alimentado, nosso sistema irá indicar
probabilidades de diagnósticos, ou seja, qual doença é mais possível que o paciente tenha
contraído.  
Após a análise o médico poderá avaliar, ou pedir um exame, para confirmar o
diagnóstico. Caso essa avaliação não confirme a doença com mais probabilidade o médico
poderá alimentar o sistema com essa informação e permitir com que as probabilidades
sejam recalculadas a partir daquela informação.  


## Modelo Conceitual 

<img src="UML - Etapa02-Page-1.png">

# Modelagem Lógica 


In [1]:
%defaultDatasource jdbc:h2:mem:db


In [2]:
CREATE TABLE Paciente (
  ID VARCHAR(9) NOT NULL,
  nome VARCHAR(50),
  nascimento VARCHAR(10),
  celular VARCHAR(7),
  localidade VARCHAR(9),
  PRIMARY KEY(ID)
) AS SELECT
    ID,
    nome,
    nascimento,
    celular, 
    localidade
FROM CSVREAD('../data/Paciente.csv');

CREATE TABLE Doenca (
  codigo VARCHAR(8) NOT NULL,
  nome VARCHAR(50),
  PRIMARY KEY(codigo)
) AS SELECT
    codigo,
    nome
FROM CSVREAD('../data/Doenca.csv');

CREATE TABLE Sintomas (
  codigo VARCHAR(8) NOT NULL,
  nome VARCHAR(50),
  PRIMARY KEY(codigo)
) AS SELECT
    codigo,
    nome
FROM CSVREAD('../data/Sintomas.csv');



In [4]:
CREATE TABLE Diagnostico (
  ID VARCHAR(10) NOT NULL,
  doencaCodigo VARCHAR(8),
  data VARCHAR(10),
  PRIMARY KEY(ID),
  FOREIGN KEY(doencaCodigo)
    REFERENCES Doenca(codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    ID,
    doencaCodigo,
    data
FROM CSVREAD('../data/Diagnostico.csv');

CREATE TABLE DoencaSintomas (
  doencaCodigo VARCHAR(8) NOT NULL,
  sintomaCodigo VARCHAR(8) NOT NULL,
  FOREIGN KEY(doencaCodigo)
    REFERENCES Doenca(codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(sintomaCodigo)
    REFERENCES Sintomas(codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    doencaCodigo,
    sintomaCodigo
FROM CSVREAD('../data/DoencaSintomas.csv');



org.h2.jdbc.JdbcSQLException:  Table "DIAGNOSTICO" already exists; SQL statement

In [5]:
CREATE TABLE PacienteDiagnostico (
  pacienteID VARCHAR(9) NOT NULL,
  diagnosticoID VARCHAR(10) NOT NULL,
  FOREIGN KEY(pacienteID)
    REFERENCES Paciente(ID)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(diagnosticoID)
    REFERENCES Diagnostico(ID)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    pacienteID,
    diagnosticoID
FROM CSVREAD('../data/PacienteDiagnostico.csv');


In [7]:

CREATE TABLE PacienteSintoma (
  Idpaciente VARCHAR(9) NOT NULL,
  CodSintoma VARCHAR(8) NOT NULL,
  FOREIGN KEY(Idpaciente)
    REFERENCES Paciente(ID)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(CodSintoma)
    REFERENCES Sintomas(codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    Idpaciente,
    CodSintoma
FROM CSVREAD('../data/PacienteSintoma.csv');




In [12]:
DROP TABLE IF EXISTS Fator;
DROP TABLE IF EXISTS FatorDoenca;
DROP TABLE IF EXISTS FatorPaciente;

CREATE TABLE Fator (
  Codigo INTEGER NOT NULL,
  Fator VARCHAR(50) NOT NULL,
  Tipo VARCHAR(50) NOT NULL,
  PRIMARY KEY(Codigo)
) AS SELECT
    Codigo,
    Fator, 
    Tipo
FROM CSVREAD('../data/Fator.csv');

CREATE TABLE FatorDoenca (
  CodFator INTEGER NOT NULL,
  Dcode VARCHAR(50) NOT NULL,
  Prob FLOAT(4) NOT NULL,
  FOREIGN KEY(CodFator)
    REFERENCES Fator(Codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(Dcode)
    REFERENCES Doenca(codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    CodFator,
    Dcode, 
    Prob
FROM CSVREAD('../data/FatorDoenca.csv');

CREATE TABLE FatorPaciente (
  CodFator INTEGER NOT NULL,
  Idpaciente VARCHAR(50) NOT NULL,
  FOREIGN KEY(CodFator)
    REFERENCES Fator(Codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(Idpaciente)
    REFERENCES Paciente(ID)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    CodFator,
    Idpaciente
FROM CSVREAD('../data/FatorPaciente.csv');



## Queries

In [9]:
SELECT * FROM Paciente WHERE Paciente.nascimento LIKE '%1990'

# Sintoma mais relatado por pacientes

In [10]:
SELECT nome FROM (SELECT S.nome, COUNT(*) contagem FROM Sintomas S, DoencaSintomas DS, Doenca D WHERE DS.sintomaCodigo = S.codigo AND DS.doencaCodigo = D.codigo GROUP BY S.nome)
WHERE contagem = (SELECT MAX(contagem) FROM (SELECT S.nome, COUNT(*) contagem FROM Sintomas S, DoencaSintomas DS, Doenca D WHERE DS.sintomaCodigo = S.codigo AND DS.doencaCodigo = D.codigo GROUP BY S.nome))

shortness of breath

# Nome dos constatados com a doença X (no caso abaixo: diabetes)

In [11]:
SELECT P.nome FROM PacienteDiagnostico PD, Diagnostico D, Doenca Do, Paciente P WHERE PD.pacienteID = P.ID AND PD.diagnosticoID = D.ID AND D.doencaCodigo = Do.codigo AND Do.nome = 'diabetes'

In [13]:
SELECT D.nome, COUNT(*) ocorrencias FROM Diagnostico Di, Doenca D, Sintomas S WHERE D.codigo = Di.doencaCodigo AND GROUP BY D.nome; 